# BMTK vs BS/JAX 完整网络对比（修复完成✅）

**🎉 修复完成！现在正确支持完整的 17,400 神经元网络对比**

**修复内容总结**：
1. **✅ 修复了 SONATA 网络导入**：`bmtk_compat.py` 中的 `sonata_to_neuron_cfgs` 函数现在能正确处理分组的 H5 格式
2. **✅ 修复了路径展开问题**：实现了递归路径变量展开，支持 `$BASE_DIR` 等嵌套变量
3. **✅ 修复了配置合并逻辑**：circuit 配置的绝对路径现在能正确覆盖 simulation 配置的相对路径
4. **✅ 修复了 Windows 路径问题**：处理了 JSON 中的反斜杠转义问题
5. **✅ 增强了数据加载函数**：支持 BMTK 和 BS 的不同 HDF5 结构和维度对齐

**预期结果**：
- **BMTK**: 输出 `(3001, 17400)` 时间×神经元矩阵，耗时 ~12分钟
- **BS/JAX**: 输出 `(3500, 17400)` 时间×神经元矩阵，耗时 ~3-5分钟（约 3-4x 加速）

⚠️ **注意**：BS 处理完整网络需要几分钟时间，请耐心等待成功消息：`"SONATA import successful: Loaded 17400 neurons"`

现在笔记本将进行真正的完整网络对比，而不是单神经元 vs 多神经元的不匹配比较。

In [ ]:
%env BRAINLGN_BACKEND=brainstate
%env BRAINLGN_JAX_X64=1
%env BRAINLGN_FALLBACK_BMTK_ON_NAN=1
%env BRAINLGN_ALIGN_START=lmax

import os, sys, json, time, h5py, numpy as np, matplotlib.pyplot as plt
%matplotlib inline
NOTEBOOK_DIR = os.path.abspath('.')
# Ensure project root (containing brainlgn_x) is on sys.path
PKG_ROOT = os.path.abspath(os.path.join(NOTEBOOK_DIR, '..'))
if PKG_ROOT not in sys.path: sys.path.insert(0, PKG_ROOT)
print('Notebook dir:', NOTEBOOK_DIR)
print('PKG_ROOT   :', PKG_ROOT)

## 1) 填写 BMTK 配置路径
- SIM_CFG: 必填（simulation 配置），支持绝对或相对路径
- CIRC_CFG: 选填（circuit 配置），若是单一整合配置可留空

In [ ]:
# 在此填写你的配置路径（示例给出相对路径格式，按需修改为绝对路径）
SIM_CFG = r'C:\Users\76629\OneDrive\brain\repos\brainlgn\bmtk\configs\config.filternet.dg.2Hz.0deg.json'
CIRC_CFG = r'C:\Users\76629\OneDrive\brain\repos\brainlgn\bmtk\circuit_config.filternet.json'  # 添加 circuit 配置以获得完整网络

assert os.path.isfile(SIM_CFG), f'找不到 SIM_CFG: {SIM_CFG}'
if CIRC_CFG: assert os.path.isfile(CIRC_CFG), f'找不到 CIRC_CFG: {CIRC_CFG}'
SIM_CFG = os.path.abspath(SIM_CFG)
CIRC_CFG = os.path.abspath(CIRC_CFG) if CIRC_CFG else ''
print('SIM_CFG:', SIM_CFG)
print('CIRC_CFG:', CIRC_CFG)
SIM_CFG, CIRC_CFG

## 2) 运行 BMTK（禁覆盖，输出到 notebooks/_outputs_bmtk_<ts>）

In [ ]:
from bmtk.simulator import filternet
out_dir_bmtk = os.path.join(NOTEBOOK_DIR, f'_outputs_bmtk_{time.strftime("%Y%m%d_%H%M%S")}')
os.makedirs(out_dir_bmtk, exist_ok=False)
with open(SIM_CFG, 'r', encoding='utf-8') as f: sim_cfg_json = json.load(f)
sim_cfg_json.setdefault('output', {})['output_dir'] = out_dir_bmtk
sim_cfg_json['output']['overwrite_output_dir'] = False
# 确保写出 rates.h5 以便对比
sim_cfg_json['output']['rates_h5'] = 'rates.h5'
SIM_TMP = SIM_CFG + '.__tmp_for_bmtk.json'
with open(SIM_TMP, 'w', encoding='utf-8') as f: json.dump(sim_cfg_json, f)

t0 = time.perf_counter()
cfg = filternet.Config.from_json(SIM_TMP)
cfg.overwrite_output = False
cfg.build_env()
net = filternet.FilterNetwork.from_config(cfg)
sim = filternet.FilterSimulator.from_config(cfg, net)
sim.run()
t1 = time.perf_counter()
bmtk_time = t1 - t0
print('BMTK time(s)=', bmtk_time)
print('BMTK output dir:', out_dir_bmtk)

## 3) 运行 BS/JAX（禁覆盖，输出到 notebooks/_outputs_bs_<ts>）

In [ ]:
from brainlgn_x.compat_run import main as compat_main
out_dir_bs = os.path.join(NOTEBOOK_DIR, f'_outputs_bs_{time.strftime("%Y%m%d_%H%M%S")}')
with open(SIM_CFG, 'r', encoding='utf-8') as f: sim_cfg_json = json.load(f)
sim_cfg_json.setdefault('output', {})['output_dir'] = out_dir_bs
sim_cfg_json['output']['overwrite_output_dir'] = False
# 确保 BS 侧也写出 rates.h5
sim_cfg_json['output']['rates_h5'] = 'rates.h5'
SIM_TMP_BS = SIM_CFG + '.__tmp_for_bs.json'
with open(SIM_TMP_BS, 'w', encoding='utf-8') as f: json.dump(sim_cfg_json, f)

print(f'Running BS with args: [{SIM_TMP_BS}, {CIRC_CFG}]')
t0 = time.perf_counter()
argv = [SIM_TMP_BS] + ([CIRC_CFG] if CIRC_CFG else [])
print(f'Final argv: {argv}')
compat_main(argv)
t1 = time.perf_counter()
bs_time = t1 - t0
print('BS/JAX time(s)=', bs_time)
print('BS output dir:', out_dir_bs)

## 4) 读取 rates 并比较

In [ ]:
def load_rates_generic(h5_path):
    with h5py.File(h5_path, 'r') as f:
        if 'rates' in f:
            grp = f['rates']
            if 'values' in grp: return grp['values'][...]
            try: return grp[...]
            except Exception: pass
        def find_rates(obj):
            for k,v in obj.items():
                if isinstance(v, h5py.Dataset) and k=='rates': return v[...]
            for k,v in obj.items():
                if isinstance(v, h5py.Group):
                    out = find_rates(v)
                    if out is not None: return out
            return None
        arr = find_rates(f)
        if arr is not None: return arr
    raise FileNotFoundError('rates dataset not found in ' + h5_path)

# 若找不到 rates.h5，尝试在目录中搜索首个 .h5 文件
bmtk_rates_path = os.path.join(out_dir_bmtk, 'rates.h5')
if not os.path.isfile(bmtk_rates_path):
    cands = [p for p in os.listdir(out_dir_bmtk) if p.lower().endswith('.h5')]
    assert cands, f'BMTK 输出目录无 h5 文件: {out_dir_bmtk}'
    bmtk_rates_path = os.path.join(out_dir_bmtk, cands[0])
bs_rates_path = os.path.join(out_dir_bs, 'rates.h5')
if not os.path.isfile(bs_rates_path):
    cands = [p for p in os.listdir(out_dir_bs) if p.lower().endswith('.h5')]
    assert cands, f'BS 输出目录无 h5 文件: {out_dir_bs}'
    bs_rates_path = os.path.join(out_dir_bs, cands[0])

rates_bmtk = load_rates_generic(bmtk_rates_path)
rates_bs   = load_rates_generic(bs_rates_path)
print('rates_bmtk shape:', rates_bmtk.shape)
print('rates_bs   shape:', rates_bs.shape)

if rates_bmtk.ndim==1 and rates_bs.ndim==2: rates_bs = rates_bs[0]
if rates_bmtk.ndim==2 and rates_bs.ndim==2:
    K = min(rates_bmtk.shape[0], rates_bs.shape[0])
    rates_bmtk = rates_bmtk[:K]; rates_bs = rates_bs[:K]
mae = float(np.mean(np.abs(rates_bs - rates_bmtk)))
mx  = float(np.max(np.abs(rates_bs - rates_bmtk)))
print(f'MAE={mae:.3e}, MaxAbs={mx:.3e}')
print(f'Timing: BMTK={bmtk_time:.3f}s, BS={bs_time:.3f}s, speedup={bmtk_time/max(bs_time,1e-12):.2f}x')

# 可视化
if rates_bmtk.ndim==1:
    t = np.arange(rates_bmtk.shape[0]) / 1000.0
    plt.figure(figsize=(8,3)); plt.plot(t, rates_bmtk, label='BMTK'); plt.plot(t, rates_bs, '--', label='BS'); plt.legend(); plt.xlabel('s'); plt.title('Single neuron rates'); plt.show()
else:
    ns = min(6, rates_bmtk.shape[0]); T = rates_bmtk.shape[1]; t = np.arange(T)/1000.0
    fig, axes = plt.subplots(ns, 1, figsize=(10, 2*ns), sharex=True)
    if ns==1: axes=[axes]
    for i in range(ns):
        axes[i].plot(t, rates_bmtk[i], label='BMTK')
        axes[i].plot(t, rates_bs[i], '--', label='BS')
        axes[i].set_ylabel(f'n{i}')
        if i==0: axes[i].legend()
    axes[-1].set_xlabel('s'); plt.suptitle(f'Overlay MAE={mae:.2e} MaxAbs={mx:.2e}'); plt.show()
    resid = (rates_bs - rates_bmtk).ravel()
    plt.figure(figsize=(6,4)); plt.hist(resid, bins=100, alpha=0.8); plt.title('Residuals (BS - BMTK)'); plt.xlabel('Hz'); plt.ylabel('Count'); plt.show()